This code is part of the lab forum at AAAI-24 on "**Harnessing Large Language Models for Planning:** A Lab on Strategies for Success and Mitigation of Pitfalls."


---


In this notebook, we will be looking at the fine-tuning procedure of CodeT5-base variant using planning problems from blocksworld.


In [1]:
!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.9 MB/s eta 0:00:00


In [2]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os
import time

# Importing the T5 modules from huggingface/transformers
from transformers import RobertaTokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console
from torch import cuda

In [3]:
#importing the training t
df = pd.read_csv("[blocksworld]data_compact - TrainDataset-classical.csv")

In [4]:
# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ),
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"),
                        title="Training Status",pad_edge=False, box=box.ASCII)

In [5]:
# Setting up the device for GPU usage
device = 'cuda' if cuda.is_available() else 'cpu'

In [6]:
class PlanningData(Dataset):
  """
  Creating a custom dataset for reading the dataset and
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long),
        'source_mask': source_mask.to(dtype=torch.long),
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [7]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [8]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask,
              max_length=200,
              num_beams=2,
              repetition_penalty=2.5,
              length_penalty=1.0,
              early_stopping=False
              )
          #print(softmax(generated_ids[1], axis=-1))
          #raise KeyboardInterrupt
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [9]:
def CodeT5Trainer(dataframe, source_text, target_text, model_params, output_dir="./" ):

  """
  CodeT5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = RobertaTokenizer.from_pretrained(model_params["MODEL"])

  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)

  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  dataframe = dataframe[[source_text,target_text]]
  display_df(dataframe.head(2))


  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation.
  train_size = 1.0
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  train_dataset.to_csv("TrainDataset.csv")
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  #val_dataset.to_csv('ValidationDataset.csv')
  train_dataset = train_dataset.reset_index(drop=True)

  console.print(f"FULL Dataset: {dataframe.shape}")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = PlanningData(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = PlanningData(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session.
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  console.log(f'[Initiating Fine Tuning]...\n')
  start_time = time.time()
  for epoch in range(model_params["TRAIN_EPOCHS"]):
      train(epoch, tokenizer, model, device, training_loader, optimizer)
  end_time = time.time()
  time_taken = end_time - start_time
  console.log("Time taken: ", time_taken, " seconds")
  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))

  console.save_text(os.path.join(output_dir,'logs.txt'))

  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [10]:
model_params={
        "MODEL":"Salesforce/codet5-base",
        "TRAIN_BATCH_SIZE":10,          # training batch size
        "VALID_BATCH_SIZE":8,          # validation batch size
        "TRAIN_EPOCHS":3,              # number of training epochs
        "VAL_EPOCHS":1,                # number of validation epochs
        "LEARNING_RATE":1e-4,          # learning rate
        "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text
        "MAX_TARGET_TEXT_LENGTH":200,   # max length of target text
        "SEED": 44                     # set seed for reproducibility
    }

In [13]:
CodeT5Trainer(dataframe=df[:10], source_text="DomainProblem", target_text="Plan", model_params=model_params, output_dir= model_params["MODEL"].replace("Salesforce/","Plansformer-")+"/")

[22:23:46] [Model]: Loading Salesforce/codet5-base...                             <ipython-input-9-197bac7c5b68>:14
                                                                                                                   

[22:23:50] [Data]: Reading data...                                                <ipython-input-9-197bac7c5b68>:25
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                      source_text                       |                       target_text                      |
|--------------------------------------------------------+--------------------------------------------------------|
|<GOAL>ontable b1, clear b1, on b2 b3, clear b2, ontable | unstack b2 b1, put-down b2, unstack b1 b4, put-down b1,|
| b3, ontable b4, clear b4<INIT>handempty, on b1 b4, on  |   unstack b4 b3, put-down b4, pick-up b2, stack b2 b3  |
| b2 b1, clear b2, ontable b3, on b4 b3<ACTION> pick-up  |                                                        |
|   <PRE> clear x, ontable x, handempty <EFFECT> not     |                                                        |
|   ontable x, not clear x, not handempty, holding x     |                                                        |
|<ACTION> put-down <PRE> holding x <EFFECT> not holding  |                                                        |
| x, clear x, handempty, ontable x <ACTION> stack <PRE>  |                                                        |
|holding x, clear y <EFFECT> not holding x, not clear y, |                                                        |
|clear x, handempty, on x y <ACTION> unstack <PRE> on x  |                                                        |
|y, clear x, handempty <EFFECT> holding x, clear y, not  |                                                        |
|          clear x, not handempty, not on x y            |                                                        |
|<GOAL>on b1 b3, clear b1, ontable b2, on b3 b2, ontable | unstack b3 b4, put-down b3, unstack b4 b2, put-down b4,|
|  b4, clear b4<INIT>handempty, ontable b1, clear b1,    |    pick-up b3, stack b3 b2, pick-up b1, stack b1 b3    |
|   ontable b2, on b3 b4, clear b3, on b4 b2<ACTION>     |                                                        |
| pick-up <PRE> clear x, ontable x, handempty <EFFECT>   |                                                        |
| not ontable x, not clear x, not handempty, holding x   |                                                        |
|<ACTION> put-down <PRE> holding x <EFFECT> not holding  |                                                        |
| x, clear x, handempty, ontable x <ACTION> stack <PRE>  |                                                        |
|holding x, clear y <EFFECT> not holding x, not clear y, |                                                        |
|clear x, handempty, on x y <ACTION> unstack <PRE> on x  |                                                        |
|y, clear x, handempty <EFFECT> holding x, clear y, not  |                                                        |
|          clear x, not handempty, not on x y            |                                                        |
+-----------------------------------------------------------------------------------------------------------------+

FULL Dataset: (10, 2)

TRAIN Dataset: (10, 2)

...

[22:23:52] [Initiating Validation]...                                             <ipython-input-9-197bac7c5b68>:91
                                                                                                                   

           [Validation Completed.]                                                <ipython-input-9-197bac7c5b68>:99
                                                                                                                   

[Model] Model saved @ Plansformer-codet5-base/model_files

[Validation] Generation on Validation data saved @ Plansformer-codet5-base/predictions.csv

[Logs] Logs saved @ Plansformer-codet5-base/logs.txt